<a href="https://colab.research.google.com/github/taerri/meal-kit-analysis/blob/main/02_%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88_%EB%B0%80%ED%82%A4%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC_Mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

konlpy 설치

In [2]:
pip install konlpy

Mecab 설치

In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Installing automake (A dependency for mecab-ko)
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,426 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  2280k      0 --:--:-- --:--:-- --:--:--  190M
./configure: line 7378: /usr/bin/file: No such file or directory
In file included from viterbi.cpp:14:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::_

In [4]:
import konlpy
import MeCab
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
import re

In [11]:
df=pd.read_csv("크롤링 결과.csv")
df.head()

,Unnamed: 0,Date,Title,Content
0,0,2017-12-27,"연말연시 ‘홈파티족’ 확산, 홈파티 ‘잇츠온’ 간편식 직접 체험해보니",요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1,1,2017-12-27,식물성·간편식·슈퍼곡물에 빠졌던 2017년,식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2,2,2017-12-27,"올해만 10여개 브랜드 출시, 가정간편식 '춘추전국시대'","야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나..."
3,3,2017-12-26,유통업계 '밀 솔루션' 경쟁 본격화,유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4,4,2017-12-19,"해먹남녀, GS리테일 ‘심플리쿡’ 판매 진행 “칼질 없이 일품요리 즐겨”",푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...


In [12]:
len(df)

5630

In [13]:
#본문 리스트 만들기
word_list=[]
word_list=df['Content']
word_list=word_list.reset_index(drop=True)

In [14]:
#기사중복 삭제
df_drop=df.drop_duplicates(['Date','Title','Content'], keep='first')

In [15]:
#index reset
df=df_drop.reset_index(drop=True)

In [16]:
#행,열 최대로 보기
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

In [17]:
df.head()
len(df)

5507

In [18]:
#기사 본문만 가져오기
content=df["Content"]
content.head()

0    요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1    식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2    야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나...
3    유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4    푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...
Name: Content, dtype: object

In [19]:
#None값 제거
drop_row = content.dropna(axis=0)
df=pd.DataFrame([drop_row])
df= df.transpose()

In [20]:
#단어 교체
content=df['Content'].map(lambda x: str(x).replace('밀 키트','밀키트'))

# **명사 추출**

In [21]:
mecab=Mecab()

In [22]:
#명사 추출
from tqdm import tqdm
nouns = [] 
for x in tqdm(content):
    try:
        nouns.append(mecab.nouns(x))
    except:
        nouns.append(["None"])

100%|██████████| 5506/5506 [00:22<00:00, 247.72it/s]


In [23]:
#명사 추출 결과값 열에 추가
konlpy_df= df.copy()
konlpy_df["ContentNoun"] = nouns

In [24]:
#json 파일로 저장
konlpy_df.to_json("mealkit_전처리.json",orient="table")

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import pickle
import re

In [26]:
corpus = pd.read_json("mealkit_전처리.json",orient="table")

# **단어 교체**

In [27]:
#단어 교체(11번 -> 11번가)
def change(text):
    text=','.join(text)
    text_list=text.split(',')
    result=[]
    for c in text_list:
        if c=='11번':
            c=c.replace('11번','11번가')
        if c=='인공지능':
            c=c.replace('인공지능','AI')
        result.append(c)
    result=','.join(result)
    return result

In [28]:
corpus['ContentNoun']=corpus['ContentNoun'].map(lambda x: change(x))

# **단어 한글자 제거**

In [29]:
def remove(text):
    text_list=text.split(',')
    result=[]
    for n in text_list:
        if len(n)>1:
            result.append(n)
    result=','.join(result)
    return result

corpus['ContentNoun']=corpus['ContentNoun'].map(lambda x: remove(x))

# **불용어 처리**

## 불용어 제거
- 사이트 이용
    - [사이트1](https://bab2min.tistory.com/544)
    - [사이트2](https://www.ranks.nl/stopwords/korean)
- 직접 제거
    - ./삭제 단어.xlsx

In [30]:
#직접 제거
stopword1 = pd.read_csv("./불용어 처리.txt",header=None,sep="\t")
stopword1 = list(stopword1[0])
len(stopword1)

139

In [31]:
stopword2 = pd.read_csv("한국어불용어100.txt",header=None,sep="\t")
stopword2 = list(stopword2[0])
len(stopword2)

100

In [32]:
stopword3 = pd.read_csv("한국어불용어100_2.txt",header=None)
stopword3 = list(stopword3[0])
len(stopword3)

675

In [33]:
stopword = list(set(stopword1 + stopword2 + stopword3))
print("불용어 개수 :",len(stopword))

불용어 개수 : 803


In [34]:
noun=corpus['ContentNoun']

In [35]:
noun = [x for x in noun if x not in stopword]
print(noun[:1])

['요리,재미,일품,분위기,거리,인파,시간,기본,식당,카페,대기,시간,고가,크리스마스,특별,메뉴,주문,가능,방침,등등,크리스마스,연말,모임,부담,생각,성비,추구,소비자,실속,의미,홈파티,확산,유통,업계,최근,식품,매출,견인,가정,편식,연말,파티,제품,한국,야쿠르트,잇츠,건강,재료,키트,중심,비밀,선물,포함,연말,럭키,박스,출시,반응,일반,레토르트,주문,마감,요리,냉장,상태,배송,때문,재료,풍미,장점,조리,때문,요리,기분,기자,이번,크리스마스이브,친구,홈파티,결정,가정,편식,체험,오전,프레시,다운로드,프라임스,테이크,림프,크림,파스타,럭키,박스,감바,주문,이틀,저녁,시간,배송,날짜,지정,시간,단위,배송,희망,시간,선택,야쿠르트,아줌마,배달,때문,달비,무료,빨간색,럭키,박스,크리스마스,선물,기분,럭키,박스,랜덤,가지,이상,선물,크림치즈,브랜드,하루,야채,콜드,브루,커피,야쿠르트,구미,젤리,선택,크리스마스,생각,우리,위장,평가,제품,인분,기준,무리,림프,크림,파스타,제외,프라임스,테이크,감바,요리,시작,프라임스,테이크,키트,고기,아스파라거스,송이버섯,토마토,양파,웨지,감자,재료,크기,손질,해당,프라임,등급,아이,프라임스,테이크,생고기,품질,기자,크기,마블링,군침,동봉,종이,레시피,스마트폰,정보,과정,완성,사진,의문,기자,키친타월,핏물,제거,허브,솔트,오일,마리네,이드,동안,재료,숙성,고기,나머지,스테이크,레시피,시간,고든,램지,빙의,소리,재촉,앞뒤,미디엄,의도,인덕션,화력,정도,감바,레시피,새우,버섯,브로콜리,베트남,고추,마늘,로즈메리,재료,과정,완성,재료,정량,손질,음식물,쓰레기,과정,재미,완성,요리,접시,향초,와인,크리스마스,캐럴,레스토랑,분위기,기자,친구,만큼,일품,고기,마지막,버터,풍미,두께,식감,고기,조화,감바,새우,생각,사이즈,올리브,오일,재료,베트남,고추,로즈메리,김동주,한국,야쿠르트,마케팅,이사,최근,가구,증가,필요,만큼,재료,손질,배송,키트,제품,인기,주문,요리,잇츠,온의,요리,방식,때문,신선도,유지,설명,전국,판매,시작,잇츠,올해,매출,목표,누적,

In [36]:
file=pd.DataFrame(noun, columns=['키워드'])
file.to_csv("단어(전처리 후).csv",encoding="utf-8-sig")

In [37]:
file.head()

,키워드
0,"요리,재미,일품,분위기,거리,인파,시간,기본,식당,카페,대기,시간,고가,크리스마스,..."
1,"식품,낭비,제로,웨이스트,발달,이국,스닉,푸드,식품,트렌드,가지,건강,중요,소비자,..."
2,"야구르트,아줌마,배달,편식,백화점,프리미엄,편식,편의점,편식,편식,춘추,전국,시대,..."
3,"유통,업계,소비자,라이프,스타일,변화,식사,가능,제품,서비스,제공,솔루션,시장,기본..."
4,"푸드,콘텐츠,커머스,플랫,해먹,남녀,운영,바이탈,힌트,코리아,대표,정지웅,테일,준비..."


In [38]:
mecab.pos('밀키트')

[('밀', 'NNG'), ('키트', 'NNG')]